# 前置部分

导库、定义变量

In [49]:
import pandas as pd
# pip install pandas openpyxl

pd.set_option('display.max_columns', None)

# 折算关系
subjects_discount = {
    '语文': 1, '数学': 1, '英语': 1, '体育': 1,
    '物理': 0.6, '化学': 0.6,
    '政治': 0.5, '历史': 0.5,
    '生物': 0.3, '地理': 0.3
}
subjects = list(subjects_discount.keys())

# 各科满分 (及格是满分的*0.6)
subjects_full = {
    '语文': 150, '数学': 150, '英语': 150, '体育': 70,
    '物理': 100, '化学': 100,
    '政治': 100, '历史': 100,
    '生物': 100, '地理': 100
}


查看数据集

In [50]:
df_info = pd.read_csv("./data/test_ksxx.csv")
df_score = pd.read_csv("./data/test_kscj.csv")
# print("df_info:\n", df_info)
# print("df_score:\n", df_score)

# print(df_info.info(), "\n")  # 43400 rows  43400 not-null
# print(df_score.info(), "\n")  # 43404 rows  43400 not-null
# print(df_info.isnull().sum(), "\n")  # 0
# print(df_score.isnull().sum(), "\n")  # 0


存在重复的考生号 (认为第二次出现为无效数据)

In [51]:
# 查看df_info中存在重复的考生号 (df_score同理)
df_info_id_count = df_info['考生号'].value_counts()
df_info_id_count_dup = df_info_id_count[df_info_id_count > 1]
df_score_id_count = df_score['考生号'].value_counts()
df_score_id_count_dup = df_score_id_count[df_score_id_count > 1]
# print(df_info_id_count_dup, df_score_id_count_dup)  # 5  # 5

# 对于df_info中重复的考生号，保留第一次出现的，删除后面的 (df_score同理)
df_info = df_info.drop_duplicates(subset='考生号', keep='first')
df_score = df_score.drop_duplicates(subset='考生号', keep='first')
# print(df_info)  # 43400 -> 43395
# print(df_score)  # 43404 -> 43399


# Q1.1

两张表考生号枚举值

In [52]:
# 列出df_info、df_score中列名为考生号的枚举值
df_info_id = df_info['考生号'].unique()
df_score_id = df_score['考生号'].unique()

# 列出df_info有的、df_score没有的考生号；列出df_score有的、df_info没有的考生号
df_info_id_only = list(set(df_info_id) - set(df_score_id))
df_score_id_only = list(set(df_score_id) - set(df_info_id))
# print(df_info_id_only, df_score_id_only)  # []  # [21951722, 63822363, 76290389, 49590743]


Q1.1：有考生信息没有考生成绩的，请将该考生成绩全部置零

In [53]:
# 不存在这样的信息，不用操作

Q1.1：如果有考生成绩没有考生信息的，请将该考生成绩记录删除

In [54]:
# 删除df_score中的df_score_id_only对应的行
df_score = df_score[~df_score['考生号'].isin(df_score_id_only)]
# print(df_score)  # 43399 -> 43395

df_score

,考生号,语文,数学,英语,物理,化学,生物,政治,历史,地理,体育
0,38096712,96,72,61,82,59,62,56,62,48,35.6
1,23257273,103,74,118,74,65,38,47,48,48,29.4
2,95989074,119,117,114,93,83,87,77,88,89,38.0
3,36935864,102,107,112,82,78,77,77,78,79,38.7
4,78065505,120,125,137,87,81,84,90,95,93,35.2
...,...,...,...,...,...,...,...,...,...,...,...
43399,44312981,100,93,69,82,59,44,20,82,91,26.6
43400,17013515,95,96,61,76,70,70,68,71,82,30.1
43401,8692931,77,83,34,67,54,62,40,55,75,38.3
43402,99451616,124,130,143,98,85,85,91,94,96,38.3


# Q1.2

查看字段：县区代码、学校代码

In [55]:
# 查看df_info中，县区代码字段的枚举值；查看df_info中，学校代码字段的枚举值
df_info_area = df_info['县区代码'].unique()
df_info_school = df_info['学校代码'].unique()
# print(len(df_info_area), len(df_info_school))  # 15  # 217
# print(df_info_area, "\n", df_info_school)

# 统计每个学校在每个县区的考生数量
df_info_group_school_area = df_info.groupby(['学校代码', '县区代码'])['考生号'].count().reset_index(name='考生数量')
# print(df_info_group_school_area)  # 376

Q1.2：部分学校出现在多个县区中，请将学校保留在考生最多的县区中，并将其它县区改为这个县区

In [56]:
# 确定每个学校最多的考生所在县区
max_students_per_school = df_info_group_school_area.loc[
    df_info_group_school_area.groupby('学校代码')['考生数量'].idxmax()]
# print(max_students_per_school)  # 217

# 创建一个映射，将学校代码映射到最多的考生所在县区代码
school_to_area_map = max_students_per_school.set_index('学校代码')['县区代码'].to_dict()
# print(len(school_to_area_map), "\n", school_to_area_map)  # 217

# 使用映射更新df_info
df_info['县区代码'] = df_info.apply(lambda row: school_to_area_map.get(row['学校代码'], row['县区代码']), axis=1)

# check
df_info_group_school_area = df_info.groupby(['学校代码', '县区代码'])['考生号'].count().reset_index(name='考生数量')
# print(df_info_group_school_area)  # 376 -> 217

df_info


,考生号,性别,县区代码,学校代码
0,60898139,女,9076,593115
1,42872499,女,160,474412
2,73504698,男,9076,593115
3,98149243,男,160,474412
4,91911749,女,160,474412
...,...,...,...,...
43395,3321431,男,9952,894151
43396,46037229,男,9952,894151
43397,71376463,男,9952,894151
43398,70375747,男,9952,205368


# Q1.3

Q1.3：折算后计算总分

In [57]:
# 折算分数
df_q13 = df_score.copy()
for subject, factor in subjects_discount.items():
    df_q13[subject] = df_score[subject] * factor
# print(df_q13)

# 将每行的语数英...字段进行累加，得到总分字段
df_q13['折算总分'] = df_q13.apply(
    lambda row: sum(row[subject] for subject in subjects), axis=1)
df_score['折算总分'] = df_q13['折算总分'].apply(lambda x: round(x, 2))

df_score

,考生号,语文,数学,英语,物理,化学,生物,政治,历史,地理,体育,折算总分
0,38096712,96,72,61,82,59,62,56,62,48,35.6,441.2
1,23257273,103,74,118,74,65,38,47,48,48,29.4,481.1
2,95989074,119,117,114,93,83,87,77,88,89,38.0,628.9
3,36935864,102,107,112,82,78,77,77,78,79,38.7,580.0
4,78065505,120,125,137,87,81,84,90,95,93,35.2,663.6
...,...,...,...,...,...,...,...,...,...,...,...,...
43399,44312981,100,93,69,82,59,44,20,82,91,26.6,464.7
43400,17013515,95,96,61,76,70,70,68,71,82,30.1,484.8
43401,8692931,77,83,34,67,54,62,40,55,75,38.3,393.5
43402,99451616,124,130,143,98,85,85,91,94,96,38.3,691.9


# Q2

合并两张表

In [58]:
df_merge_info_score = pd.merge(df_info, df_score, on='考生号')
# print(len(df_info), len(df_score), len(df_merge_info_score))  # 43395  # 43395  # 43395

# 以csv形式输出中间结果
df_info.to_csv("./tmp/df_info.csv", index=False)
df_score.to_csv("./tmp/df_score.csv", index=False)
df_merge_info_score.to_csv("./tmp/df_merge_info_score.csv", index=False)


# 在df_merge_info_score，当subjects字段都为0时，认为是缺考，其缺考字段设置为1，否则设置为0
df_merge_info_score['缺考'] = df_merge_info_score.apply(
    lambda row: 1 if sum(row[subject] for subject in subjects) == 0 else 0, axis=1)
df_merge_info_score.to_csv("./tmp/df_merge_info_score.csv", index=False)

# check 查看缺考
df_q22_missing = df_merge_info_score[df_merge_info_score['缺考'] == 1]
# print(df_q22_missing)  # 609


每个考生的语数英三科均分率、十科均分率

In [59]:
# 计算每个考生的语数英总分、所有科目成绩累加
df_merge_info_score['语数英总分'] = df_merge_info_score[subjects[:3]].sum(axis=1)
# 计算每个考生的语数英均分率(450；要求以百分比形式，保留两位小数)
df_merge_info_score['语数英三科均分率(%)'] = df_merge_info_score.apply(
    lambda row: round(row['语数英总分'] / 450 * 100, 2), axis=1)
# 计算每个考生的所有科目均分率(800；要求以百分比形式，保留两位小数)
df_merge_info_score['十科均分率(%)'] = df_merge_info_score.apply(
    lambda row: round(row['折算总分'] / 800 * 100, 2), axis=1)
df_merge_info_score.to_csv("./tmp/df_merge_info_score.csv", index=False)


每个考生的优秀情况

In [60]:
# 在df_merge_info_score，判断考生是否优秀(折算总分字段>=640)，优秀为1，否则为0
df_merge_info_score['是否优秀'] = df_merge_info_score.apply(lambda row: 1 if row['折算总分'] >= 640 else 0, axis=1)
df_merge_info_score.to_csv("./tmp/df_merge_info_score.csv", index=False)


每个考生的及格情况

In [61]:
# 在df_merge_info_score，分别在各个科目中，判断考生是否及格，及格为1，否则为0
for subject, full in subjects_full.items():
    df_merge_info_score[subject + '是否及格'] = df_merge_info_score.apply(
        lambda row: 1 if row[subject] >= full * 0.6 else 0, axis=1)

# df_merge_info_score，对[subject + '是否及格']的各个字段求和，得到一个考生的及格次数
df_merge_info_score['单科及格次数'] = df_merge_info_score[
    list(subject + '是否及格' for subject in subjects_full.keys())].sum(axis=1)
df_merge_info_score.to_csv("./tmp/df_merge_info_score.csv", index=False)

df_merge_info_score


,考生号,性别,县区代码,学校代码,语文,数学,英语,物理,化学,生物,政治,历史,地理,体育,折算总分,缺考,语数英总分,语数英三科均分率(%),十科均分率(%),是否优秀,语文是否及格,数学是否及格,英语是否及格,体育是否及格,物理是否及格,化学是否及格,政治是否及格,历史是否及格,生物是否及格,地理是否及格,单科及格次数
0,60898139,女,9076,593115,72,43,48,55,41,32,31,61,44,28.9,318.3,0,163,36.22,39.79,0,0,0,0,0,0,0,0,1,0,0,1
1,42872499,女,160,474412,113,114,132,86,77,88,87,90,86,38.9,636.4,0,359,79.78,79.55,0,1,1,1,0,1,1,1,1,1,1,9
2,73504698,男,9076,593115,14,55,39,35,0,0,12,79,48,36.4,225.3,0,108,24.00,28.16,0,0,0,0,0,0,0,0,1,0,0,1
3,98149243,男,160,474412,100,112,61,81,67,85,79,85,91,39.5,536.1,0,273,60.67,67.01,0,1,1,0,0,1,1,1,1,1,1,8
4,91911749,女,160,474412,103,109,122,83,54,66,71,75,72,38.4,569.0,0,334,74.22,71.12,0,1,1,1,0,1,0,1,1,1,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43390,3321431,男,9952,894151,86,42,42,75,69,29,19,62,62,32.3,356.5,0,170,37.78,44.56,0,0,0,0,0,1,1,0,1,0,1,4
43391,46037229,男,9952,894151,0,0,0,0,0,0,0,20,21,0.0,16.3,0,0,0.00,2.04,0,0,0,0,0,0,0,0,0,0,0,0
43392,71376463,男,9952,894151,0,0,0,0,0,0,0,12,22,0.0,12.6,0,0,0.00,1.57,0,0,0,0,0,0,0,0,0,0,0,0
43393,70375747,男,9952,205368,47,51,41,35,32,20,13,39,45,29.1,253.8,0,139,30.89,31.73,0,0,0,0,0,0,0,0,0,0,0,0


# Q2.2

每个县区的考生数、与考数、与考率

In [62]:
# 对于每个县区，计算考生数
df_q22_1 = df_merge_info_score['县区代码'].value_counts().reset_index()
df_q22_1.columns = ['县区代码', '考生数']
# 对于每个县区，计算非缺考人数
every_area_missing = df_merge_info_score.groupby('县区代码')['缺考'].sum().reset_index()
every_area_missing.columns = ['县区代码', '缺考人数']
# print(df_q22_1, "\n", every_area_missing)

# 对于每个县区，与考数 = 考生数 - 缺考人数
df_q22 = pd.merge(df_q22_1, every_area_missing, on='县区代码', how='left')
df_q22['与考数'] = df_q22['考生数'] - df_q22['缺考人数']
# 对于每个学校，计算与考率 (与考数除以考生数，要求以百分比形式，保留两位小数)
df_q22["与考率(%)"] = df_q22.apply(lambda row: round(row['与考数'] / row['考生数'] * 100, 2), axis=1)
df_q22.to_csv("./tmp/df_q22.csv", index=False)


每个县区的语数英均分率、十科均分率

In [63]:
# 计算每个县区的语数英均分率、所有科目均分率
df_q22_2 = df_merge_info_score.groupby('县区代码')['语数英三科均分率(%)'].mean().reset_index(name='语数英三科均分率(%)')
df_q22_3 = df_merge_info_score.groupby('县区代码')['十科均分率(%)'].mean().reset_index(name='十科均分率(%)')
# 保留两位小数
df_q22_2['语数英三科均分率(%)'] = df_q22_2['语数英三科均分率(%)'].apply(lambda x: round(x, 2))
df_q22_3['十科均分率(%)'] = df_q22_3['十科均分率(%)'].apply(lambda x: round(x, 2))
# print(df_q22_2，"\n", df_q22_3)

df_q22 = pd.merge(df_q22, df_q22_2, on='县区代码', how='left')
df_q22 = pd.merge(df_q22, df_q22_3, on='县区代码', how='left')
df_q22.to_csv("./tmp/df_q22.csv", index=False)

每个县区的优秀率、单科及格率

In [64]:
# 计算每个县区的优秀率
df_q22_4 = df_merge_info_score.groupby('县区代码')['是否优秀'].mean().reset_index(name='优秀率(%)')
df_q22_4['优秀率(%)'] = df_q22_4['优秀率(%)'].apply(lambda x: round(x * 100, 2))
# print(df_q22_4)
df_q22 = pd.merge(df_q22, df_q22_4, on='县区代码', how='left')
df_q22.to_csv("./tmp/df_q22.csv", index=False)

# 计算每个县区的及格人次
df_q22_5 = df_merge_info_score.groupby('县区代码')['单科及格次数'].sum().reset_index(name='单科及格人次')
# print(df_q22_5)
df_q22 = pd.merge(df_q22, df_q22_5, on='县区代码', how='left')
# 计算每个县区的及格率 (及格人次除以考生数*10，要求以百分比形式，保留两位小数)
df_q22['单科及格率(%)'] = df_q22.apply(lambda row: round(row['单科及格人次'] / (row['与考数'] * 10) * 100, 2), axis=1)
df_q22.to_csv("./tmp/df_q22.csv", index=False)

每个县区的四率综合比、排名

In [65]:
# 对df_q22，四率综合比：（语数英三科均分率+十科均分率）* 0.3 + （优秀率 + 单科及格率）* 0.2。（要求以百分比形式，保留两位小数）
df_q22['四率综合比(%)'] = df_q22.apply(
    lambda row: round(
        (row['语数英三科均分率(%)'] + row['十科均分率(%)']) * 0.3 + (row['优秀率(%)'] + row['单科及格率(%)']) * 0.2, 2),
    axis=1)
# 对df_q22，按综合四率排序
df_q22['排名'] = df_q22['四率综合比(%)'].rank(ascending=False).astype(int)
df_q22 = df_q22.sort_values(by='四率综合比(%)', ascending=False)
df_q22.to_csv("./tmp/df_q22.csv", index=False)

输出结果

In [66]:
# 整理列
df_q22 = df_q22.drop(columns=['缺考人数', '单科及格人次'])
df_q22.to_csv("./tmp/df_q22.csv", index=False)

df_q22

,县区代码,考生数,与考数,与考率(%),语数英三科均分率(%),十科均分率(%),优秀率(%),单科及格率(%),四率综合比(%),排名
4,5670,3912,3906,99.85,70.56,70.63,40.62,72.27,64.94,1
14,864,618,618,100.00,73.15,70.74,35.44,71.18,64.49,2
1,9952,5694,5668,99.54,63.76,64.23,31.59,61.92,57.10,3
8,9852,1888,1874,99.26,64.08,64.60,27.70,63.84,56.91,4
6,1614,2156,2154,99.91,63.09,63.25,17.95,60.99,53.69,5
7,7764,2005,1985,99.00,61.21,61.08,26.13,57.95,53.50,6
12,160,1355,1352,99.78,62.67,62.55,16.83,59.17,52.77,7
11,7451,1469,1461,99.46,59.57,60.47,15.86,57.36,50.66,8
0,9076,7413,7301,98.49,56.94,57.89,22.06,54.69,49.80,9
10,173,1482,1438,97.03,58.97,58.77,14.98,56.95,49.71,10


# Q2.3

每个学校的考生数、与考数、与考率

In [67]:
# 对于每个学校，计算考生数
df_q23_1 = df_merge_info_score['学校代码'].value_counts().reset_index()
df_q23_1.columns = ['学校代码', '考生数']
# 对于每个学校，计算非缺考人数
every_school_missing = df_merge_info_score.groupby('学校代码')['缺考'].sum().reset_index()
every_school_missing.columns = ['学校代码', '缺考人数']
# print(df_q23_1, every_school_missing)

# 对于每个学校，与考数 = 考生数 - 缺考人数
df_q23 = pd.merge(df_q23_1, every_school_missing, on='学校代码', how='left')
df_q23['与考数'] = df_q23['考生数'] - df_q23['缺考人数']
# 对于每个学校，计算与考率 (与考数除以考生数，要求以百分比形式，保留两位小数)
df_q23["与考率(%)"] = df_q23.apply(lambda row: round(row['与考数'] / row['考生数'] * 100, 2), axis=1)
df_q23.to_csv("./tmp/df_q23.csv", index=False)


每个学校的语数英三科均分率、十科均分率

In [68]:
# 计算每个学校的语数英均分率、所有科目均分率
df_q23_2 = df_merge_info_score.groupby('学校代码')['语数英三科均分率(%)'].mean().reset_index(name='语数英三科均分率(%)')
df_q23_3 = df_merge_info_score.groupby('学校代码')['十科均分率(%)'].mean().reset_index(name='十科均分率(%)')
# 保留两位小数
df_q23_2['语数英三科均分率(%)'] = df_q23_2['语数英三科均分率(%)'].apply(lambda x: round(x, 2))
df_q23_3['十科均分率(%)'] = df_q23_3['十科均分率(%)'].apply(lambda x: round(x, 2))
# print(df_q23_2, df_q23_3)

df_q23 = pd.merge(df_q23, df_q23_2, on='学校代码', how='left')
df_q23 = pd.merge(df_q23, df_q23_3, on='学校代码', how='left')
df_q22.to_csv("./tmp/df_q23.csv", index=False)

每个学校的优秀率、单科及格率

In [69]:
# 计算每个学校的优秀率
df_q23_4 = df_merge_info_score.groupby('学校代码')['是否优秀'].mean().reset_index(name='优秀率(%)')
df_q23_4['优秀率(%)'] = df_q23_4['优秀率(%)'].apply(lambda x: round(x * 100, 2))
# print(df_q23_4)
df_q23 = pd.merge(df_q23, df_q23_4, on='学校代码', how='left')
df_q23.to_csv("./tmp/df_q23.csv", index=False)

# 计算每个学校的及格人次
df_q23_5 = df_merge_info_score.groupby('学校代码')['单科及格次数'].sum().reset_index(name='单科及格人次')
# print(df_q22_5)
df_q23 = pd.merge(df_q23, df_q23_5, on='学校代码', how='left')
# 计算每个学校的及格率 (及格人次除以考生数*10，要求以百分比形式，保留两位小数)
df_q23['单科及格率(%)'] = df_q23.apply(lambda row: round(row['单科及格人次'] / (row['与考数'] * 10) * 100, 2), axis=1)
df_q23.to_csv("./tmp/df_q23.csv", index=False)

每个学校的四率综合比、排名

In [70]:
# 对df_q23，四率综合比：（语数英三科均分率+十科均分率）* 0.3 + （优秀率 + 单科及格率）* 0.2。（要求以百分比形式，保留两位小数）
df_q23['四率综合比(%)'] = df_q23.apply(
    lambda row: round(
        (row['语数英三科均分率(%)'] + row['十科均分率(%)']) * 0.3 + (row['优秀率(%)'] + row['单科及格率(%)']) * 0.2, 2),
    axis=1)
# 对df_q23，按综合四率排序
df_q23['排名'] = df_q23['四率综合比(%)'].rank(ascending=False).astype(int)
df_q23 = df_q23.sort_values(by='四率综合比(%)', ascending=False)
df_q23.to_csv("./tmp/df_q23.csv", index=False)

输出结果

In [71]:
# 创建县区代码和学校代码的映射关系
df_q23_6 = df_info[["县区代码", "学校代码"]]
map_for_area = {}
for index, row in df_q23_6.iterrows():
    map_for_area[row["学校代码"]] = row["县区代码"]
# 县区代码的枚举值
area_enum = df_q23_6["县区代码"].unique()
# print(area_enum)  # [9076  160  173 4678 7764 9952 9852 5670  864 8717 1995 1614 7451 3281 992]

# 将map_for_area转为df
df_map_for_area = pd.DataFrame(map_for_area.items(), columns=["学校代码", "县区代码"])
# print(df_map_for_area)

# 整理列
df_q23 = df_q23.drop(columns=['缺考人数', '单科及格人次'])
# df_q23左连接df_info，得到每个学校的县区代码
df_q23 = pd.merge(df_q23, df_map_for_area, on='学校代码', how='left')
df_q23.to_csv("./tmp/df_q23.csv", index=False)

# df_23按照县区代码字段进行分组，分别存储到许多个df中
df_q23_result = {}
for area in area_enum:
    df_q23_result[area] = df_q23[df_q23["县区代码"] == area]
# 将最后一系列df_result分别保存到多个csv中
for area in area_enum:
    df_q23_result[area].to_csv("./tmp/df_q23_result_" + str(area) + ".csv", index=False)

df_q23_result[area_enum[0]]

,学校代码,考生数,与考数,与考率(%),语数英三科均分率(%),十科均分率(%),优秀率(%),单科及格率(%),四率综合比(%),排名,县区代码
8,764387,780,780,100.00,78.37,78.22,52.56,85.38,74.56,9,9076
9,566364,285,285,100.00,77.87,77.48,58.95,80.74,74.54,10,9076
14,837865,98,98,100.00,75.43,75.67,42.86,81.94,70.29,15,9076
24,473436,282,280,99.29,68.26,69.28,34.40,71.32,62.41,25,9076
44,645225,47,47,100.00,64.70,66.14,6.38,70.00,54.53,45,9076
53,104759,35,35,100.00,62.83,63.60,0.00,66.29,51.19,54,9076
62,832819,242,238,98.35,57.24,57.92,23.97,53.87,50.12,63,9076
63,321665,308,304,98.70,57.78,58.62,21.10,54.64,50.07,64,9076
70,770207,130,124,95.38,56.55,57.05,19.23,57.74,49.47,71,9076
76,435224,90,89,98.89,56.41,58.40,16.67,54.83,48.74,77,9076


保存在excel

In [72]:
df_q22.to_excel('zlfx.xlsx', sheet_name='县区', index=False)
with pd.ExcelWriter('zlfx.xlsx', mode='a', if_sheet_exists='overlay') as writer:
    for area, df in df_q23_result.items():
        df.to_excel(writer, sheet_name=str(area), index=False)